In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Load train and test data
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

print("Train data shape:", train_df.shape)
print("Test data shape:", test_df.shape)
print("\nTrain data info:")
print(train_df.info())
print("\nMissing values in train data:")
print(train_df.isnull().sum())
print("\nMissing values in test data:")
print(test_df.isnull().sum())

# Check the target variable distribution
plt.figure(figsize=(10, 6))
sns.histplot(train_df['song_popularity'], bins=30, kde=True)
plt.title('Distribution of Song Popularity')
plt.show()

def preprocess_data(df, is_train=True, imputer_dict=None, scaler_dict=None):
    """
    Preprocess the data with proper handling for train/test sets
    """
    df_processed = df.copy()
    
    # Separate features and target if it's training data
    if is_train:
        X = df_processed.drop('song_popularity', axis=1)
        y = df_processed['song_popularity']
    else:
        X = df_processed
        y = None
    
    # Handle missing values
    numerical_features = ['song_duration_ms', 'acousticness', 'danceability', 'energy', 
                         'instrumentalness', 'liveness', 'loudness', 'speechiness', 
                         'tempo', 'audio_valence']
    
    categorical_features = ['key', 'time_signature', 'audio_mode']
    
    # Initialize imputers if not provided
    if imputer_dict is None:
        imputer_dict = {
            'num_imputer': KNNImputer(n_neighbors=5),
            'cat_imputer': SimpleImputer(strategy='most_frequent')
        }
    
    # Apply imputation
    X[numerical_features] = imputer_dict['num_imputer'].fit_transform(X[numerical_features]) if is_train else imputer_dict['num_imputer'].transform(X[numerical_features])
    X[categorical_features] = imputer_dict['cat_imputer'].fit_transform(X[categorical_features]) if is_train else imputer_dict['cat_imputer'].transform(X[categorical_features])
    
    # Feature engineering
    X = create_features(X)
    
    # Scale numerical features
    numerical_features_extended = [f for f in X.columns if f not in categorical_features + ['id']]
    
    if scaler_dict is None:
        scaler_dict = {
            'scaler': StandardScaler()
        }
    
    X_scaled = scaler_dict['scaler'].fit_transform(X[numerical_features_extended]) if is_train else scaler_dict['scaler'].transform(X[numerical_features_extended])
    
    # Create final DataFrame
    X_final = pd.DataFrame(X_scaled, columns=numerical_features_extended, index=X.index)
    X_final[categorical_features] = X[categorical_features].values
    
    return X_final, y, imputer_dict, scaler_dict

def create_features(X):
    """Create additional features"""
    X_featured = X.copy()
    
    # Interaction features
    X_featured['energy_danceability'] = X_featured['energy'] * X_featured['danceability']
    X_featured['valence_energy'] = X_featured['audio_valence'] * X_featured['energy']
    X_featured['acoustic_energy'] = X_featured['acousticness'] * X_featured['energy']
    X_featured['dance_valence'] = X_featured['danceability'] * X_featured['audio_valence']
    
    # Duration features
    X_featured['duration_minutes'] = X_featured['song_duration_ms'] / 60000
    X_featured['duration_seconds'] = X_featured['song_duration_ms'] / 1000
    
    # Loudness features
    X_featured['loudness_abs'] = abs(X_featured['loudness'])
    
    # Tempo features
    X_featured['tempo_category'] = pd.cut(X_featured['tempo'], 
                                        bins=[0, 60, 100, 140, 200, 300],
                                        labels=[0, 1, 2, 3, 4])
    X_featured['tempo_category'] = X_featured['tempo_category'].astype('category')
    
    # Energy ratio features
    X_featured['energy_loudness_ratio'] = X_featured['energy'] / (abs(X_featured['loudness']) + 1)
    
    return X_featured

# Preprocess training data
X_train, y_train, imputer_dict, scaler_dict = preprocess_data(train_df, is_train=True)

# Preprocess test data using the same imputers and scaler from training
X_test, y_test, _, _ = preprocess_data(test_df, is_train=False, 
                                      imputer_dict=imputer_dict, 
                                      scaler_dict=scaler_dict)

print("Processed train features shape:", X_train.shape)
print("Processed test features shape:", X_test.shape)

# Define multiple models with different algorithms
models = {
    'Random Forest': RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=200, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=200, random_state=42, n_jobs=-1),
    'LightGBM': LGBMRegressor(n_estimators=200, random_state=42, n_jobs=-1),
    'CatBoost': CatBoostRegressor(n_estimators=200, random_state=42, verbose=0),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.01, random_state=42),
    'ElasticNet': ElasticNet(alpha=0.01, l1_ratio=0.5, random_state=42),
    'SVR': SVR(kernel='rbf', C=1.0)
}

# Train and evaluate each model
results = {}
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    
    # Predict on training data for validation
    y_pred = model.predict(X_train)
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_pred)
    train_mse = mean_squared_error(y_train, y_pred)
    train_mae = mean_absolute_error(y_train, y_pred)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2', n_jobs=-1)
    
    results[name] = {
        'train_r2': train_r2,
        'train_mse': train_mse,
        'train_mae': train_mae,
        'cv_mean_r2': cv_scores.mean(),
        'cv_std_r2': cv_scores.std()
    }
    
    print(f"{name}: Train R² = {train_r2:.4f}, CV R² = {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

# Display results
results_df = pd.DataFrame(results).T
print("\nModel Performance Summary:")
print(results_df.sort_values('cv_mean_r2', ascending=False))

# Create base models for ensemble
base_models = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)),
    ('xgb', XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1)),
    ('lgbm', LGBMRegressor(n_estimators=100, random_state=42, n_jobs=-1)),
    ('gb', GradientBoostingRegressor(n_estimators=100, random_state=42))
]

# Voting Ensemble
voting_ensemble = VotingRegressor(estimators=base_models, n_jobs=-1)
voting_ensemble.fit(X_train, y_train)

# Stacking Ensemble
stacking_ensemble = StackingRegressor(
    estimators=base_models,
    final_estimator=GradientBoostingRegressor(n_estimators=50, random_state=42),
    n_jobs=-1
)
stacking_ensemble.fit(X_train, y_train)

# Evaluate ensembles
ensemble_results = {}
for name, ensemble in [('Voting', voting_ensemble), ('Stacking', stacking_ensemble)]:
    y_pred = ensemble.predict(X_train)
    cv_scores = cross_val_score(ensemble, X_train, y_train, cv=5, scoring='r2', n_jobs=-1)
    
    ensemble_results[name] = {
        'train_r2': r2_score(y_train, y_pred),
        'cv_mean_r2': cv_scores.mean(),
        'cv_std_r2': cv_scores.std()
    }
    print(f"{name} Ensemble: R² = {r2_score(y_train, y_pred):.4f}, CV R² = {cv_scores.mean():.4f}")

# Add to results
ensemble_results_df = pd.DataFrame(ensemble_results).T
results_df = pd.concat([results_df, ensemble_results_df])

# Let's tune the best performing model (usually XGBoost or LightGBM)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Use RandomizedSearchCV for faster tuning
xgb_tuned = RandomizedSearchCV(
    XGBRegressor(random_state=42, n_jobs=-1),
    param_grid,
    n_iter=50,
    cv=3,
    scoring='r2',
    random_state=42,
    n_jobs=-1
)

xgb_tuned.fit(X_train, y_train)

print("Best XGBoost parameters:", xgb_tuned.best_params_)
print("Best XGBoost score:", xgb_tuned.best_score_)

# Train final model with best parameters
final_model = XGBRegressor(
    **xgb_tuned.best_params_,
    random_state=42,
    n_jobs=-1
)

final_model.fit(X_train, y_train)

# Make predictions on test set
test_predictions = final_model.predict(X_test)

# Create submission file
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'song_popularity': test_predictions
})

# Save predictions
file_output_name = "submission_1.csv"
submission_df.to_csv(file_output_name, index=False)
print(f"Submission file '{file_output_name}' created!")

# Evaluate final model (if test has actual values)
if 'song_popularity' in test_df.columns:
    test_actual = test_df['song_popularity']
    test_r2 = r2_score(test_actual, test_predictions)
    test_mse = mean_squared_error(test_actual, test_predictions)
    print(f"Final Test Performance: R² = {test_r2:.4f}, MSE = {test_mse:.4f}")

# Analyze feature importance
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 10))
sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
plt.title('Top 20 Feature Importances')
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.show()

print("Top 10 most important features:")
print(feature_importance.head(10))

# Create a summary of key insights
print("="*50)
print("MODEL INSIGHTS AND RECOMMENDATIONS")
print("="*50)

print("\n1. Top Predictive Features:")
for i, row in feature_importance.head(5).iterrows():
    print(f"   {row['feature']}: {row['importance']:.4f}")

print("\n2. Model Performance Summary:")
best_models = results_df.sort_values('cv_mean_r2', ascending=False).head(3)
for model_name in best_models.index:
    score = best_models.loc[model_name, 'cv_mean_r2']
    print(f"   {model_name}: R² = {score:.4f}")

print("\n3. Key Factors for Song Popularity:")
print("   - Danceability and energy are typically strong predictors")
print("   - Loudness and tempo often contribute significantly")
print("   - Valence (musical positiveness) can influence popularity")
print("   - Duration and acousticness may have complex relationships")

print("\n4. Recommendations for Improvement:")
print("   - Try additional feature engineering")
print("   - Experiment with different scaling methods")
print("   - Consider neural networks for complex patterns")
print("   - Use feature selection to reduce dimensionality")

In [ ]:
# Open ./submission_1.py, and if song_popularity  > 0.37, put it 1, else 0
import pandas as pd

df = pd.read_csv('./submission_1.csv')
df['song_popularity'] = df['song_popularity'].apply(lambda x: 1 if x > 0.365052 else 0)
df.to_csv('./submission_4.csv', index=False)